In [1]:
import pandas as pd
data = pd.read_csv("CP_count_PUMA.csv")
data

,smiles,1_1,10_19,10_20,100_277,100_278,104_291,105_355,105_369,105_361,...,Cells_Neighbors_FirstClosestObjectNumber_Adjacent,Cells_Neighbors_SecondClosestObjectNumber_5,Cells_Neighbors_SecondClosestObjectNumber_Adjacent,Cells_Parent_Nuclei,Cytoplasm_Number_Object_Number,Cytoplasm_Parent_Cells,Cytoplasm_Parent_Nuclei,Nuclei_Neighbors_FirstClosestObjectNumber_1,Nuclei_Neighbors_SecondClosestObjectNumber_1,Nuclei_Number_Object_Number
0,CCOC(=O)c1ccc(NC(=S)N2CCSC2c2ccc(OC)cc2)cc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.765625,0.890625,0.890625,1.328125,1.328125,1.328125,1.328125,0.960938,1.320312,1.328125
1,O=C(CC1NC(=O)NC1=O)Nc1cccc2ccccc12,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.078125,2.718750,2.718750,2.503906,2.503906,2.503906,2.503906,3.242188,2.578125,2.503906
2,CC(Nc1nc(nc2ccccc12)N1CCCC1)c1ccccc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-7.765625,-8.140625,-8.140625,-8.250000,-8.250000,-8.250000,-8.250000,-7.710938,-7.546875,-8.250000
3,CCCC(Oc1ccc(Br)cc1)c1nc2c3cc(OC)c(OC)cc3nc(S)n2n1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-7.320312,-7.828125,-7.828125,-7.796875,-7.804688,-7.796875,-7.796875,-7.859375,-7.695312,-7.804688
4,CC1=C(C(NC(=O)N1)c1ccc(F)cc1)C(=O)OCc1ccc2OCOc2c1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-10.953125,-10.390625,-10.390625,-10.960938,-10.960938,-10.960938,-10.960938,-11.406250,-10.781250,-10.960938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16165,COC(=O)[C@H]1[C@H](CO)[C@H]2Cn3c(=O)c(\C=C\C)c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.039062,-2.031250,-2.031250,-1.550781,-1.550781,-1.550781,-1.550781,-1.546875,-2.945312,-1.550781
16166,C\C=C\c1ccc2n(C[C@H]3[C@H](CO)[C@H](N(C)[C@@H]...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.851562,-0.359375,-0.359375,-0.835938,-0.835938,-0.835938,-0.835938,-1.328125,-0.804688,-0.835938
16167,C\C=C\c1ccc2n(C[C@@H]3[C@@H](CO)[C@@H](N(C)[C@...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-3.851562,-3.359375,-3.359375,-3.835938,-3.835938,-3.835938,-3.835938,-3.328125,-4.054688,-3.835938
16168,C\C=C\c1ccc2n(C[C@H]3[C@H](CO)[C@@H](C(=O)N[C@...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-4.039062,-2.460938,-2.460938,-3.300781,-3.300781,-3.300781,-3.300781,-3.382812,-2.937500,-3.300781


In [2]:
assays_list = data.columns.to_list()[1:-13]
len(assays_list)

270

In [3]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
import pandas as pd
from tqdm import tqdm


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier


def load_and_preprocess_data(fold):
    train_path = f"PUMA/predictions/chemical_cv{fold}/assay_matrix_discrete_train_scaff.csv"
    test_path = f"PUMA/predictions/chemical_cv{fold}/assay_matrix_discrete_test_scaff.csv"

    train_data = pd.read_csv(train_path)[["smiles"]]
    train_data = pd.merge(train_data, data, on="smiles")
    
    test_data = pd.read_csv(test_path)[["smiles"]]
    test_data = pd.merge(test_data, data, on="smiles")
    
    return train_data, test_data



In [35]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import roc_auc_score
import pandas as pd
from joblib import Parallel, delayed
from tqdm import tqdm
from sklearn.utils import class_weight
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from xgboost import XGBClassifier

# Function to train and return the model for a given fold
def train_model_for_fold(fold):
    train_data, _ = load_and_preprocess_data(fold)
    
    X_train_combined = pd.DataFrame()
    y_train_combined = pd.Series()

    for assay in assays_list:
        train_data_assay = train_data[~train_data[assay].isna()][[assay, "Cells_Number_Object_Number"]]
        X_train_combined = pd.concat([X_train_combined, train_data_assay[["Cells_Number_Object_Number"]]], axis=0)
        y_train_combined = pd.concat([y_train_combined, train_data_assay[assay].astype(int)], axis=0)

    # Calculate class weights (balanced)
    class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_combined), y=y_train_combined)
    class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
    # print(class_weight_dict)

    # Calculate scale_pos_weight based on class weights
    scale_pos_weight = class_weight_dict[0] / class_weight_dict[1]
    # print("scale_pos_weight: ", scale_pos_weight)

    # Define the parameter grid with additional hyperparameters and wider ranges
    param_grid = {
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.3],
        'n_estimators': [100, 200],
        'scale_pos_weight': [scale_pos_weight, 1],
        'n_jobs': [-1]
        }

    # Initialize the XGBoost classifier
    xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42, n_jobs=-1)

    stratified_kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

    # Perform RandomizedSearchCV
    random_search = RandomizedSearchCV(
            estimator=xgb_clf,
            param_distributions=param_grid,
            n_iter=10,  # Number of parameter settings that are sampled
            scoring='f1',  # Metric for imbalanced dataset (you can also try 'roc_auc' or 'average_precision')
            cv=stratified_kfold,  # 5-fold cross-validation
            n_jobs=10,  # Use all processors
            verbose=0,
            random_state=42
        )

    # Fit the model on training data
    random_search.fit(X_train_combined, y_train_combined)

    # Best estimator
    best_clf = random_search.best_estimator_
    # print(f"Best parameters: {random_search.best_params_}")
        
    # clf = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=5, n_jobs=-1)
    best_clf.fit(X_train_combined, y_train_combined)
    
    return best_clf

# Parallelize the training of models across all folds
models_per_fold = Parallel(n_jobs=-1)(delayed(train_model_for_fold)(fold) for fold in tqdm(folds))

# Function to test and return AUC score for a given fold and model
def test_model_for_fold(idx, fold, clf):
    train_data, test_data = load_and_preprocess_data(fold)
    auc_scores = []
    
    stratified_kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

    for assay in assays_list:
        train_data_assay = train_data[~train_data[assay].isna()][[assay, "Cells_Number_Object_Number"]]
        X_train = train_data_assay[["Cells_Number_Object_Number"]]
        y_train = train_data_assay[assay].astype(int)

        active_count = (y_train == 1).sum()
        inactive_count = (y_train == 0).sum()

        if active_count >= 6 and inactive_count >= 6:
         
            # Calculate class weights (balanced)
            class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
            class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
            # print(class_weight_dict)

            # Calculate scale_pos_weight based on class weights
            scale_pos_weight = class_weight_dict[0] / class_weight_dict[1]
            # print("scale_pos_weight: ", scale_pos_weight)

            # Define the parameter grid with additional hyperparameters and wider ranges
            param_grid = {
                'max_depth': [3, 5, 7],
                'learning_rate': [0.01, 0.1, 0.3],
                'n_estimators': [100, 200],
                'scale_pos_weight': [scale_pos_weight, 1],
                'n_jobs': [-1]
            }

            # Initialize the XGBoost classifier
            xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42, n_jobs=-1)

            # Perform RandomizedSearchCV
            random_search = RandomizedSearchCV(
                    estimator=xgb_clf,
                    param_distributions=param_grid,
                    n_iter=10,  # Number of parameter settings that are sampled
                    scoring='f1',  # Metric for imbalanced dataset (you can also try 'roc_auc' or 'average_precision')
                    cv=stratified_kfold,  # 5-fold cross-validation
                    n_jobs=10,  # Use all processors
                    verbose=0,
                    random_state=42
                )

            # Fit the model on training data
            random_search.fit(X_train, y_train)

            # Best estimator
            clf = random_search.best_estimator_
            # print(f"Best parameters: {random_search.best_params_}")

            # clf = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=5, n_jobs=-1)
            clf.fit(X_train, y_train)

        test_data_assay = test_data[~test_data[assay].isna()][[assay, "Cells_Number_Object_Number"]]
        X_test = test_data_assay[["Cells_Number_Object_Number"]]
        y_test = test_data_assay[assay].astype(int)

        if len(y_test) == 0:
            print(f"Skipping assay {assay} in fold {fold} due to no test data")
            continue

        y_pred_proba = clf.predict_proba(X_test)[:, 1]
        auc_score = np.nan
        try:
            auc_score = roc_auc_score(y_test, y_pred_proba)
        except:
            pass

        auc_scores.append({"Fold": fold, "Task": assay, "AUC": auc_score})

    return auc_scores

# Parallelize the testing of models across all folds
results = Parallel(n_jobs=-1)(delayed(test_model_for_fold)(idx, fold, models_per_fold[idx]) for idx, fold in enumerate(tqdm(folds)))

# Combine all results into a DataFrame
all_auc_scores = [score for fold_results in results for score in fold_results]

# Convert the AUC results to a DataFrame for convenience
auc_df = pd.DataFrame(all_auc_scores)

# Save the DataFrame to a CSV file
auc_df.to_csv("RF_auc_scores_across_folds_individual_models.csv", index=False)

# Display the AUC DataFrame
auc_df


100%|██████████| 5/5 [00:00<00:00, 4898.74it/s]


,Fold,Task,AUC
0,0,1_1,0.761905
1,0,10_19,NaN
2,0,10_20,NaN
3,0,100_277,0.812500
4,0,100_278,0.785714
...,...,...,...
1345,4,9_18,0.350000
1346,4,90_256,0.553571
1347,4,91_257,0.750000
1348,4,92_258,0.750000


In [36]:
mean_auc = auc_df.groupby("Task").mean().reset_index(drop=False)
mean_auc

,Task,AUC
0,100_277,0.678611
1,100_278,0.790079
2,104_289,0.529487
3,104_291,0.572115
4,104_295,0.428571
...,...,...
265,92_258,0.701190
266,93_259,0.474990
267,9_14,0.000000
268,9_17,0.714286


In [37]:
len(mean_auc[mean_auc["AUC"]<0.20])

7

In [38]:
len(mean_auc[mean_auc["AUC"]==1])

5

In [39]:
len(mean_auc[mean_auc["AUC"]>0.90])

9

In [40]:
len(mean_auc[mean_auc["AUC"]>0.80])

28

In [41]:
mean_auc.to_csv("RF_scaffold_mean_AUCSeal_individual_models.csv")

In [42]:
mean_auc.mean()

Task         inf
AUC     0.579739
dtype: float64

In [43]:
mean_auc.std()

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


AUC    0.169317
dtype: float64

In [44]:
median_auc = auc_df.groupby("Task").median().reset_index(drop=False)
median_auc.to_csv("RF_scaffold_median_AUCSeal_individual_models.csv")

In [45]:
len(median_auc[median_auc["AUC"]==1])

10

In [46]:
len(median_auc[median_auc["AUC"]>0.90])

14